# Politics and the stock market

In [ ]:
# hide
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import pandas as pd

from skfin.metrics import sharpe_ratio
from skfin.plot import *
from skfin.datasets_ import load_kf_returns

from IPython.display import Image, display

import logging 
logging.getLogger("skfin.dataloaders.cache").setLevel(level=logging.CRITICAL)

ret = load_kf_returns(filename="F-F_Research_Data_Factors")["Monthly"]

In this section, we explore the relationship between political control in the United States government and stock market performance, focusing on the phenomenon known as the "Government Puzzle." We first discuss the "Presidential Puzzle"—the observation that U.S. stock market excess returns have historically been higher under Democratic presidents. 

We then examine whether the alignment or division of political power across the executive and legislative branches (i.e., United vs. Divided government) has an even greater impact on financial markets and economic growth. 

## The Presidential puzzle and the division of the government

In [ ]:
# hide
display(Image("images/santa_clara_valkanov.png", width=600))

In [ ]:
# hide
display(Image("images/papamichalis_ryu_wilson.png", width=600))

The figure below from Papamichalis, Ryu and Wilson (2024) shows a two-way sort of average annual excess stock returns by both the president's party and whether the government is united or divided, using post-war data (1927–2020). The main takeaway is that Republican and Democratic presidents deliver strong stock market returns when they have unified control of government, while Republican presidents under divided government are associated with particularly weak market performance—demonstrating that the government cycle, not just presidential party, is key to explaining stock market outcomes.

In [ ]:
# hide
display(Image("images/papamichalis_ryu_wilson_table.png", width=600))

## Politics over century of data

In [ ]:
from skfin.dataloaders.constants.dates import load_us_politics_dates

end_date = "2022-10-01"  # Midterm election during Biden's term

cols = ["Presidency", "House", "Senate"]
cols_ = ret.columns
government_ffill_shift = (
    lambda x, cols: x[cols]
    .shift(1)
    .ffill()
    .join(x.drop(cols, axis=1).dropna(), how="right")
)
df = (
    load_us_politics_dates()
    .resample("MS")
    .last()
    .join(ret, how="outer")
    .pipe(government_ffill_shift, cols)
    .assign(
        united_government=lambda x: x[cols].apply(
            lambda row: row.nunique() == 1, axis=1
        ),
        democratic_president=lambda x: x["Presidency"] == "Democratic",
    )
    .loc[:end_date]
)

We first look wether there is a monthly seasonality in returns. 

In [ ]:
bar(
    df.groupby(lambda x: x.month)["Mkt-RF"].apply(sharpe_ratio, num_period_per_year=12),
    sort=False,
    title="Annualized sharpe ratio by month",
)

In particular, the bar chart below shows that there is no strong imbalance in the number of months in each subcategories. 

In [ ]:
vars = ["democratic_president", "united_government"]

fig, ax = plt.subplots(1, 2, figsize=(18, 6))
fig.suptitle("Number of months")
for i, c in enumerate(vars):
    bar(df.groupby(c)["RF"].count(), ax=ax[i], title=c)

The two graphs below essentially replicates the results in Santa-Clara and Valkanov (2003) and Papamichalis et al. (2024): higher Sharpe ratio (and gain) for democratic presidents and unified governments. 

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 6))
fig.suptitle("Sharpe ratio")
for i, c in enumerate(vars):
    bar(
        df.groupby(c)["Mkt-RF"].apply(sharpe_ratio, num_period_per_year=12),
        ax=ax[i],
        title=c,
    )

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 6))
fig.suptitle("Annualized mean")
for i, c in enumerate(vars):
    bar(df.groupby(c)["Mkt-RF"].mean().mul(12), ax=ax[i], title=c)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 6))
fig.suptitle("Annualized standard deviation")
for i, c in enumerate(vars):
    bar(df.groupby(c)["Mkt-RF"].std().mul(np.sqrt(12)), ax=ax[i], title=c)

In [ ]:
q = "Mkt-RF"
c = "united_government"

dfs = {
    c: df.assign(**{c: lambda x: x[c].apply(str)})
    .set_index(c, append=True)[q]
    .unstack()
    .fillna(0)
    .join(ret[q])
    for c in vars
}

fig, ax = plt.subplots(1, 2, figsize=(20, 8))
for i, (k, v) in enumerate(dfs.items()):
    line(v, cumsum=True, title=k, ax=ax[i], loc="best")